In [82]:
# importing 
import pandas as pd
import sqlite3

In [83]:
# Load Orders (CSV) 
orders = pd.read_csv(r"C:\Users\thaneshwar\Downloads\orders (1).csv")
print("Orders Data Loaded:", orders.shape)

Orders Data Loaded: (10000, 6)


In [84]:
# Load Users (JSON) 
users = pd.read_json(r"C:\Users\thaneshwar\Downloads\users.json")
print("Users Data Loaded:", users.shape)

Users Data Loaded: (3000, 4)


In [85]:
for col in users.columns:
    if "member" in col.lower() or "plan" in col.lower(): 
        users = users.rename(columns={col: "membership_type"}) 
        break

In [86]:
conn = sqlite3.connect(":memory:")

In [87]:
with open(r"C:\Users\thaneshwar\Downloads\restaurants.sql", "r") as f: 
    sql_script = f.read()

In [88]:
conn.executescript(sql_script)

In [89]:
# Read restaurant master data
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

In [90]:
print("Restaurants Data Loaded:", restaurants.shape) 
print(restaurants.head())

Restaurants Data Loaded: (500, 4)
   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [91]:
conn.close()

In [92]:
merged_df = orders.merge(users, on="user_id", how="left").merge(restaurants, on="restaurant_id", how="left")


In [93]:
# Save Final Dataset 
merged_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [94]:
print(merged_df.columns.tolist())

['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_x', 'name', 'city', 'membership_type', 'restaurant_name_y', 'cuisine', 'rating']


In [95]:
print(merged_df.head())


   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership_type  \
0               New Foods Chinese  User_2508  Hyderabad         Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune         Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai            Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore            Gold   
4       Royal Eatery South Indian  User_1064       Pune         Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3 

In [96]:
# Fix order amount column
if "total_amount" in merged_df.columns:
    merged_df = merged_df.rename(columns={"total_amount": "amount"})
elif "order_value" in merged_df.columns:
    merged_df = merged_df.rename(columns={"order_value": "amount"})
elif "price" in merged_df.columns:
    merged_df = merged_df.rename(columns={"price": "amount"})

# Fix city column (sometimes user_city or restaurant_city)
if "user_city" in merged_df.columns:
    merged_df = merged_df.rename(columns={"user_city": "city"})
elif "restaurant_city" in merged_df.columns:
    merged_df = merged_df.rename(columns={"restaurant_city": "city"})


In [97]:
print(merged_df[['city','amount']].head())

        city   amount
0  Hyderabad   842.97
1       Pune   546.68
2    Chennai   163.93
3  Bangalore  1155.97
4       Pune  1321.91


# Analysis

In [98]:
# Q1: City with highest total revenue from Gold members
q1 = merged_df[merged_df['membership_type'] == 'Gold']
q1 = q1.groupby('city')['amount'].sum().sort_values(ascending=False)
print("Q1:", q1.head(1))



Q1: city
Chennai    1080909.79
Name: amount, dtype: float64


In [99]:
# Q2: Cuisine with highest average order value 
q2 = merged_df.groupby('cuisine')['amount'].mean().sort_values(ascending=False) 
print("Q2:", q2.head(1))

Q2: cuisine
Mexican    808.021344
Name: amount, dtype: float64


In [100]:
# Q3: Distinct users with total orders > 1000 
user_totals = merged_df.groupby('user_id')['amount'].sum() 
q3 = user_totals[user_totals > 1000].nunique() 
print("Q3: Distinct users >", q3)

Q3: Distinct users > 2543


In [101]:
# Q4: Restaurant rating range with highest revenue 
def rating_bucket(r):
    if 3.0 <= r <= 3.5:
        return "3.0–3.5" 
    elif 3.6 <= r <= 4.0:
        return "3.6–4.0" 
    elif 4.1 <= r <= 4.5:
        return "4.1–4.5" 
    elif 4.6 <= r <= 5.0:
        return "4.6–5.0" 
    else: 
        return "Other"
merged_df['rating_range'] = merged_df['rating'].apply(rating_bucket) 
q4 = merged_df.groupby('rating_range')['amount'].sum().sort_values(ascending=False) 
print("Q4:", q4.head(1))

Q4: rating_range
4.6–5.0    2197030.75
Name: amount, dtype: float64


In [102]:
# Q5: Among Gold members, city with highest average order value 
q5 = merged_df[merged_df['membership_type'] == 'Gold'] 
q5 = q5.groupby('city')['amount'].mean().sort_values(ascending=False) 
print("Q5:", q5.head(1))

Q5: city
Chennai    808.45908
Name: amount, dtype: float64


In [103]:
# Q6: Cuisine with lowest distinct restaurants but significant revenue 
q6_counts = merged_df.groupby('cuisine')['restaurant_id'].nunique()
q6_revenue = merged_df.groupby('cuisine')['amount'].sum() 
q6 = pd.DataFrame({'restaurants': q6_counts, 'revenue': q6_revenue}).sort_values(by='restaurants') 
print("Q6:", q6)

Q6:          restaurants     revenue
cuisine                         
Chinese          120  1930504.65
Indian           126  1971412.58
Italian          126  2024203.80
Mexican          128  2085503.09


In [104]:
# Q7: Percentage of total orders by Gold members 
gold_orders = merged_df[merged_df['membership_type'] == 'Gold'].shape[0]
total_orders = merged_df.shape[0] 
q7 = round((gold_orders / total_orders) * 100) 
print("Q7:", q7, "%")

Q7: 50 %


In [105]:
# Q8: Restaurant with highest avg order value but <20 orders
q8 = merged_df.groupby('restaurant_name_x').agg(
    avg_order=('amount','mean'),
    total_orders=('order_id','count')
)
q8 = q8[q8['total_orders'] < 20].sort_values('avg_order', ascending=False)
print("Q8:", q8.head(1))


Q8:                             avg_order  total_orders
restaurant_name_x                                  
Hotel Dhaba Multicuisine  1040.222308            13


In [106]:
q9 = merged_df.groupby(['membership_type','cuisine'])['amount'].sum().sort_values(ascending=False)
print("Q9:", q9.head(1))


Q9: membership_type  cuisine
Regular          Mexican    1072943.3
Name: amount, dtype: float64


In [107]:
# Q10: Quarter with highest revenue
# Parse dates correctly (day-month-year format)
merged_df['order_date'] = pd.to_datetime(merged_df['order_date'], format="%d-%m-%Y")

# Extract quarter
merged_df['quarter'] = merged_df['order_date'].dt.quarter

# Group by quarter and sum revenue
q10 = merged_df.groupby('quarter')['amount'].sum().sort_values(ascending=False)

# Map quarter numbers to labels
quarter_labels = {1: "Q1 (Jan–Mar)", 2: "Q2 (Apr–Jun)", 3: "Q3 (Jul–Sep)", 4: "Q4 (Oct–Dec)"}
q10.index = q10.index.map(quarter_labels)

print("Q10: Quarter with highest revenue")
print(q10.head(1))


Q10: Quarter with highest revenue
quarter
Q3 (Jul–Sep)    2037385.1
Name: amount, dtype: float64


In [108]:
# How many total orders were placed by users with Gold membership?
gold_orders = merged_df[merged_df['membership_type'] == 'Gold'].shape[0]
print("Total orders by Gold members:", gold_orders)

Total orders by Gold members: 4987


In [109]:
# What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
hyd_revenue = merged_df[merged_df['city'] == 'Hyderabad']['amount'].sum()
print("Total revenue from Hyderabad (rounded):", round(hyd_revenue))


Total revenue from Hyderabad (rounded): 1889367


In [110]:
# How many distinct users placed at least one order?
distinct_users = merged_df['user_id'].nunique()
print("Distinct users who placed at least one order:", distinct_users)

Distinct users who placed at least one order: 2883


In [111]:
# What is the average order value (rounded to 2 decimals) for Gold members?
avg_gold_order = merged_df[merged_df['membership_type'] == 'Gold']['amount'].mean()
print("Average order value for Gold members:", round(avg_gold_order, 2))

Average order value for Gold members: 797.15


In [112]:
# How many orders were placed for restaurants with rating ≥ 4.5?
high_rating_orders = merged_df[merged_df['rating'] >= 4.5].shape[0]
print("Orders placed for restaurants with rating ≥ 4.5:", high_rating_orders)

Orders placed for restaurants with rating ≥ 4.5: 3374


In [113]:
# How many orders were placed in the top revenue city among Gold members only?
gold_city_revenue = merged_df[merged_df['membership_type'] == 'Gold'].groupby('city')['amount'].sum()
top_gold_city = gold_city_revenue.sort_values(ascending=False).index[0]

orders_in_top_gold_city = merged_df[(merged_df['membership_type'] == 'Gold') & (merged_df['city'] == top_gold_city)].shape[0]
print("Orders placed in top revenue city among Gold members:", orders_in_top_gold_city)

Orders placed in top revenue city among Gold members: 1337
